In [1]:
import torch
import pandas as pd
import os

In [2]:
!pip install clean-text
!pip install langdetect
#!pip install validators

In [3]:
from gptzero.model import GPT2PPL
from cleantext import clean
from langdetect import detect
#import validators

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


In [4]:
import re
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

In [5]:
def isurl(input):   
    regex = re.compile(
        r'^(?:http|ftp)s?://' # http:// or https://
        r'(?:(?:[A-Z0-9](?:[A-Z0-9-]{0,61}[A-Z0-9])?\.)+(?:[A-Z]{2,6}\.?|[A-Z0-9-]{2,}\.?)|' #domain...
        r'localhost|' #localhost...
        r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})' # ...or ip
        r'(?::\d+)?' # optional port
        r'(?:/?|[/?]\S+)$', re.IGNORECASE)
    return re.match(regex, input) is not None 

def printline():
    print("------------------------------------------")
    
def infer(sentence):
    sentence1 = sentence
    sentence = remove_emojis(sentence)
    sentence = clean(sentence, no_emoji=True)
    if sentence != sentence1:
        print(sentence1, sentence)
    model = GPT2PPL()
    output = model(sentence)
    return output[0]["label"]

def enhanced_detect(input):
    printline()
    print(input)
    printline()
    return detect(input)

In [6]:
df = pd.read_csv("full_frame.csv", sep='\t', encoding='utf-8')

In [7]:
df.head()

,utterances,human
0,Yeah but that's not the detail I am looking fo...,1
1,"Mais tu veux dire ""on est"", si même si tu voul...",1
2,U should check out the stuff that Jordan Peter...,0
3,And here are the rules of the game.\n\n>The ob...,1
4,That's not the point.,1


In [ ]:
testsent = "The fitnessgram pacer test is an aerobic multistage test designed to get progressively more difficult as it continues."
infer(testsent)
df.describe()


In [ ]:
#Clean the Data
df = df.dropna(subset=['utterances'])
df = df[df.utterances != '']
df["isurl"] = df["utterances"].map(isurl)
df = df[~df.isurl]
df["language"] = df["utterances"].map(enhanced_detect)
#df.describe()

In [ ]:
df["predictions"] = df["utterances"].map(infer)